## Extracting eye-tracking data outputs from SR

In [1]:
# Import modules

import os
import pandas as pd
from pathlib import Path
import numpy as np
import glob
import shutil
#import mne

In [2]:
# Function to format ID
def format_id(id_str):
    parts = id_str.split('_')
    #if len(parts) != 2:
        #raise ValueError("Invalid ID format. Should be 'XXX_XX' or 'XXXX_XX' or 'XXX_X' or 'XXXX_XX'.")
    
    numeric_part = parts[0]
    letter_part = parts[1]
    
    #if not numeric_part.isdigit():
     #   print("Numeric part before '_' should consist of digits only.")
    
    
    # Ensure numeric part is 4 digits long by padding with zeros if necessary
    padded_numeric_part = numeric_part.zfill(4)
    
    formatted_id = f"{padded_numeric_part}{letter_part}"
    
    return formatted_id

# Fuction to fix experiment builder ID's so that they match the ET ID's
def eb_id_transform(file):
    file = file.upper()
    file = file.replace('Q', "")

    if "_" not in file:
        # Add "_" right before the first letter from the end 
        # Find the index of the first alphanumeric character
        for i, char in enumerate(file):
            if char.isalpha():
                break      
        # Insert "_" before the first alphanumeric character found
        file = file[:i] + '_' + file[i:]
    
        
    # add 0's to the end of the file name to make it 4 digits
    file = format_id(file)
    return file

# Function to check if file name contains task information
def has_task_info(file_name, task_info):
    return task_info in file_name

## Function to check if file name contains task information
def process_participants(task_folder, eeg_q1k_subjects_df, missing_eeg, et_subjects, transformed_et):
    """
    Process participant folders to track missing EEG data.
    
    :param task_folder: Path to the current task folder.
    :param eeg_q1k_subjects_df: DataFrame containing EEG subjects data.
    :param missing_eeg: List to store participants with missing EEG data.
    :param et_subjects: List to store participant names.
    :param transformed_et: List to store transformed participant IDs.
    """    
    for participant in os.listdir(task_folder):
        participant_folder = os.path.join(task_folder, participant)
        
        et_subjects.append(participant)
        transformed_id = eb_id_transform(participant)
        transformed_et.append(transformed_id)
        
        if transformed_id in eeg_q1k_subjects_df.et_ID.values:
            # Retrieve new participant ID if available
            new_participant = eeg_q1k_subjects_df.loc[eeg_q1k_subjects_df['et_ID'] == transformed_id, 'q1k_ID'].values
            if new_participant.size > 0:
                new_id = new_participant[0]
        else:
            missing_eeg.append(participant)
    return new_id

# Function to copy and paste participant files
def process_participant_files(task, participant, participant_folder, transformed_id, new_participant, final_output_dir):
    # Process .edf and .txt files in one pass
    for file_name in os.listdir(participant_folder):
        if file_name.endswith('.edf'):
            #new_file_name = f"{new_participant}_{task}.{file_name.split('.')[-1]}"
            #new_file_name = f"{new_participant}_{task}.asc"
            new_file_name = f"{transformed_id}_{task}.asc"
            source_path = os.path.join(participant_folder, file_name)
            destination_path = os.path.join(final_output_dir, new_participant, new_file_name)
            #shutil.copy(source_path, destination_path)
            print('new_file_name: ' + new_file_name)
            print('source_path: ' + source_path)
            print('destination_path: ' + destination_path)
            command = "edf2asc -y -input " + source_path + " " + destination_path
            print(command)
            !{command}

        #elif file_name.endswith('.txt') and has_task_info(file_name, task):
        #    new_file_name = f"{new_participant}_{task}.{file_name.split('.')[-1]}"
        #    source_path = os.path.join(participant_folder, file_name)
        #    destination_path = os.path.join(final_output_dir, new_participant, new_file_name)
        #    shutil.copy(source_path, destination_path)

## Create reference file to map Q1K IDs to eye tracking data

In [ ]:
eeg_q1k_subjects= []
truncated_eeg_q1k_subjects = []
family_id_subjects = []
site = "HSJ" #'MHC' or 'HSJ'

In [4]:
# Create a list of tasks  
#tasks = ['GO', 'NSP', 'AS', 'PLR','VS','FSP','REST', 'SSAEP', 'SSVEP','TMMN','TO']
task_id_in = 'NSP'
task_id_out = 'NSP'

In [5]:
#glob.glob(f"../../../../../../Sharing/CHUSJ-Q1K-PILOT/experimental/{site}/sourcedata/eeg/*")
#glob.glob(f"../../sourcedata/{site}/et/*")
glob.glob(f"../../sourcedata/{site}/eeg/*")


['../../sourcedata/MHC/eeg/Q1K_MHC_200181_P',
 '../../sourcedata/MHC/eeg/Q1K_MHC_20068_M1',
 '../../sourcedata/MHC/eeg/Q1K_MHC_20042_P',
 '../../sourcedata/MHC/eeg/Q1K_MHC_200186_M1',
 '../../sourcedata/MHC/eeg/Q1K_MHC_200171_M1',
 '../../sourcedata/MHC/eeg/Q1K_MHC_200218_M1',
 '../../sourcedata/MHC/eeg/Q1K_MHC_200183_S1',
 '../../sourcedata/MHC/eeg/Q1K_MHC_200248_P',
 '../../sourcedata/MHC/eeg/Q1K_MHC_200275_F1',
 '../../sourcedata/MHC/eeg/Q1K_MHC_200223_M1',
 '../../sourcedata/MHC/eeg/Q1K_MHC_200200_P',
 '../../sourcedata/MHC/eeg/Q1K_MHC_200218_P',
 '../../sourcedata/MHC/eeg/Q1K_MHC_200200_F1',
 '../../sourcedata/MHC/eeg/Q1K_MHC_200181_M1',
 '../../sourcedata/MHC/eeg/Q1K_MHC_200196_M1',
 '../../sourcedata/MHC/eeg/Q1K_MHC_200223_P',
 '../../sourcedata/MHC/eeg/Q1K_MHC_200171_S1',
 '../../sourcedata/MHC/eeg/Q1K_MHC_200196_S2',
 '../../sourcedata/MHC/eeg/Q1K_MHC_20068_S1',
 '../../sourcedata/MHC/eeg/Q1K_MHC_200183_M1',
 '../../sourcedata/MHC/eeg/Q1K_MHC_200183_P',
 '../../sourcedata/MHC/

In [6]:
# Create a list of all files in the EEG folders

#for site in sites: 
    #for file in glob.glob(f"../../../../../../Sharing/CHUSJ-Q1K-PILOT/experimental/{site}/sourcedata/eeg/*"):
for file in glob.glob(f"../../sourcedata/{site}/eeg/*"):    
    subject_id = file.split('/')[-1]
    # Skip sessions that have already been processed
    print(subject_id)
    #eeg_q1k_subjects.append(subject_id)
    if "1025" in subject_id:
        eeg_q1k_subjects.append(subject_id)
        truncated_id=subject_id.split('1025')[1][1:]
        truncated_id=format_id(truncated_id)
        truncated_eeg_q1k_subjects.append(truncated_id)
        family_id = truncated_id.split('_')[0]
        family_id_subjects.append(family_id)
    #    print(subject_id)
    elif "1525" in subject_id:
        eeg_q1k_subjects.append(subject_id)
        truncated_id=subject_id.split('1525')[1][1:]
        truncated_id=format_id(truncated_id)
        truncated_eeg_q1k_subjects.append(truncated_id)
        family_id = truncated_id.split('_')[0]
        family_id_subjects.append(family_id)
    #elif "HSJ" in subject_id:
    elif subject_id.startswith("Q1K_HSJ_100"):
        eeg_q1k_subjects.append(subject_id)
        truncated_id=subject_id.split('Q1K_HSJ_100')[1]
        truncated_id=format_id(truncated_id)
        truncated_eeg_q1k_subjects.append(truncated_id)
        family_id = truncated_id.split('_')[0]
        family_id_subjects.append(family_id)
    elif "MHC" in subject_id:
        eeg_q1k_subjects.append(subject_id)
        truncated_id=subject_id.split('Q1K_MHC_200')[1]
        truncated_id=format_id(truncated_id)
        truncated_eeg_q1k_subjects.append(truncated_id)
        family_id = truncated_id.split('_')[0]
        family_id_subjects.append(family_id)

    #truncated_eeg_q1k_subjects.append(truncated_id)
    print(truncated_id)
    length = len(subject_id)
    #family_id = truncated_id.split('_')[0]
    #family_id_subjects.append(family_id)


Q1K_MHC_200181_P
0181P
Q1K_MHC_20068_M1
0068M1
Q1K_MHC_20042_P
0042P
Q1K_MHC_200186_M1
0186M1
Q1K_MHC_200171_M1
0171M1
Q1K_MHC_200218_M1
0218M1
Q1K_MHC_200183_S1
0183S1
Q1K_MHC_200248_P
0248P
Q1K_MHC_200275_F1
0275F1
Q1K_MHC_200223_M1
0223M1
Q1K_MHC_200200_P
0200P
Q1K_MHC_200218_P
0218P
Q1K_MHC_200200_F1
0200F1
Q1K_MHC_200181_M1
0181M1
Q1K_MHC_200196_M1
0196M1
Q1K_MHC_200223_P
0223P
Q1K_MHC_200171_S1
0171S1
Q1K_MHC_200196_S2
0196S2
Q1K_MHC_20068_S1
0068S1
Q1K_MHC_200183_M1
0183M1
Q1K_MHC_200183_P
0183P
Q1K_MHC_200265_M1
0265M1
Q1K_MHC_200196_P
0196P
Q1K_MHC_200248_M1
0248M1
Q1K_MHC_200275_P
0275P
Q1K_MHC_200181_F1
0181F1
Q1K_MHC_200216_P
0216P
Q1K_MHC_20068_S3
0068S3
Q1K_MHC_200200_M1
0200M1
Q1K_MHC_200265_P
0265P
Q1K_MHC_200216_M1
0216M1
Q1K_MHC_200171_P
0171P
Q1K_MHC_200179_P
0179P
Q1K_MHC_200281_P
0281P
Q1K_MHC_200281_M1
0281M1
Q1K_MHC_20068_F1
0068F1
Q1K_MHC_200200_S1
0200S1
Q1K_MHC_200179_M1
0179M1
Q1K_MHC_200171_F1
0171F1
Q1K_MHC_200248_S1
0248S1
Q1K_MHC_200186_F1
0186F1


In [7]:
eeg_q1k_subjects_df = pd.DataFrame({'q1k_ID': eeg_q1k_subjects, 'et_ID': truncated_eeg_q1k_subjects,
                                    'family_ID': family_id_subjects})
# Add 0s to the et_ID ID to make it 4 digits
#eeg_q1k_subjects_df['et_ID'] = eeg_q1k_subjects_df['et_ID'].apply(lambda x: format_id(x))

In [8]:
eeg_q1k_subjects_df

,q1k_ID,et_ID,family_ID
0,Q1K_MHC_200181_P,0181P,0181P
1,Q1K_MHC_20068_M1,0068M1,0068M1
2,Q1K_MHC_20042_P,0042P,0042P
3,Q1K_MHC_200186_M1,0186M1,0186M1
4,Q1K_MHC_200171_M1,0171M1,0171M1
5,Q1K_MHC_200218_M1,0218M1,0218M1
6,Q1K_MHC_200183_S1,0183S1,0183S1
7,Q1K_MHC_200248_P,0248P,0248P
8,Q1K_MHC_200275_F1,0275F1,0275F1
9,Q1K_MHC_200223_M1,0223M1,0223M1


In [9]:
print("There are a total of" , len(eeg_q1k_subjects_df.et_ID.unique()), "unique participants")

There are a total of 41 unique participants


In [10]:
glob.glob(f'../../../../Sharing/CHUSJ-Q1K-PILOT/experimental/{site}/sourcedata/et/Q1K*_2.2.299/*_{task_id_in}_*/results/*')

['../../../../Sharing/CHUSJ-Q1K-PILOT/experimental/MHC/sourcedata/et/Q1K_EB_2.2.299/6_Q1K_ACAR_NSP_pfp_v1_02_deploy/results/CBG',
 '../../../../Sharing/CHUSJ-Q1K-PILOT/experimental/MHC/sourcedata/et/Q1K_EB_2.2.299/6_Q1K_ACAR_NSP_pfp_v1_02_deploy/results/Q248_P',
 '../../../../Sharing/CHUSJ-Q1K-PILOT/experimental/MHC/sourcedata/et/Q1K_EB_2.2.299/6_Q1K_ACAR_NSP_pfp_v1_02_deploy/results/281_M1',
 '../../../../Sharing/CHUSJ-Q1K-PILOT/experimental/MHC/sourcedata/et/Q1K_EB_2.2.299/6_Q1K_ACAR_NSP_pfp_v1_02_deploy/results/AD',
 '../../../../Sharing/CHUSJ-Q1K-PILOT/experimental/MHC/sourcedata/et/Q1K_EB_2.2.299/6_Q1K_ACAR_NSP_pfp_v1_02_deploy/results/JDM',
 '../../../../Sharing/CHUSJ-Q1K-PILOT/experimental/MHC/sourcedata/et/Q1K_EB_2.2.299/6_Q1K_ACAR_NSP_pfp_v1_02_deploy/results/281_P',
 '../../../../Sharing/CHUSJ-Q1K-PILOT/experimental/MHC/sourcedata/et/Q1K_EB_2.2.299/6_Q1K_ACAR_NSP_pfp_v1_02_deploy/results/Q223_M1',
 '../../../../Sharing/CHUSJ-Q1K-PILOT/experimental/MHC/sourcedata/et/Q1K_EB_2.2

In [11]:
#glob.glob(f"../../../../Sharing/CHUSJ-Q1K-PILOT/experimental/{site}/sourcedata/et/Q1K_2.2.299/*")

root_dir = '../../../../Sharing/CHUSJ-Q1K-PILOT/experimental/'
output_dir = f'../../sourcedata/{site}/et/'
et_dir= os.path.join(root_dir, f"{site}/sourcedata/et/Q1K*_2.2.299/")
glob.glob(os.path.join(et_dir, f"*_{task_id_in}_*/results/*"))

['../../../../Sharing/CHUSJ-Q1K-PILOT/experimental/MHC/sourcedata/et/Q1K_EB_2.2.299/6_Q1K_ACAR_NSP_pfp_v1_02_deploy/results/CBG',
 '../../../../Sharing/CHUSJ-Q1K-PILOT/experimental/MHC/sourcedata/et/Q1K_EB_2.2.299/6_Q1K_ACAR_NSP_pfp_v1_02_deploy/results/Q248_P',
 '../../../../Sharing/CHUSJ-Q1K-PILOT/experimental/MHC/sourcedata/et/Q1K_EB_2.2.299/6_Q1K_ACAR_NSP_pfp_v1_02_deploy/results/281_M1',
 '../../../../Sharing/CHUSJ-Q1K-PILOT/experimental/MHC/sourcedata/et/Q1K_EB_2.2.299/6_Q1K_ACAR_NSP_pfp_v1_02_deploy/results/AD',
 '../../../../Sharing/CHUSJ-Q1K-PILOT/experimental/MHC/sourcedata/et/Q1K_EB_2.2.299/6_Q1K_ACAR_NSP_pfp_v1_02_deploy/results/JDM',
 '../../../../Sharing/CHUSJ-Q1K-PILOT/experimental/MHC/sourcedata/et/Q1K_EB_2.2.299/6_Q1K_ACAR_NSP_pfp_v1_02_deploy/results/281_P',
 '../../../../Sharing/CHUSJ-Q1K-PILOT/experimental/MHC/sourcedata/et/Q1K_EB_2.2.299/6_Q1K_ACAR_NSP_pfp_v1_02_deploy/results/Q223_M1',
 '../../../../Sharing/CHUSJ-Q1K-PILOT/experimental/MHC/sourcedata/et/Q1K_EB_2.2

In [12]:
task_folders = glob.glob(os.path.join(et_dir, f"*_{task_id_in}_*/results/*"))
task_folders

['../../../../Sharing/CHUSJ-Q1K-PILOT/experimental/MHC/sourcedata/et/Q1K_EB_2.2.299/6_Q1K_ACAR_NSP_pfp_v1_02_deploy/results/CBG',
 '../../../../Sharing/CHUSJ-Q1K-PILOT/experimental/MHC/sourcedata/et/Q1K_EB_2.2.299/6_Q1K_ACAR_NSP_pfp_v1_02_deploy/results/Q248_P',
 '../../../../Sharing/CHUSJ-Q1K-PILOT/experimental/MHC/sourcedata/et/Q1K_EB_2.2.299/6_Q1K_ACAR_NSP_pfp_v1_02_deploy/results/281_M1',
 '../../../../Sharing/CHUSJ-Q1K-PILOT/experimental/MHC/sourcedata/et/Q1K_EB_2.2.299/6_Q1K_ACAR_NSP_pfp_v1_02_deploy/results/AD',
 '../../../../Sharing/CHUSJ-Q1K-PILOT/experimental/MHC/sourcedata/et/Q1K_EB_2.2.299/6_Q1K_ACAR_NSP_pfp_v1_02_deploy/results/JDM',
 '../../../../Sharing/CHUSJ-Q1K-PILOT/experimental/MHC/sourcedata/et/Q1K_EB_2.2.299/6_Q1K_ACAR_NSP_pfp_v1_02_deploy/results/281_P',
 '../../../../Sharing/CHUSJ-Q1K-PILOT/experimental/MHC/sourcedata/et/Q1K_EB_2.2.299/6_Q1K_ACAR_NSP_pfp_v1_02_deploy/results/Q223_M1',
 '../../../../Sharing/CHUSJ-Q1K-PILOT/experimental/MHC/sourcedata/et/Q1K_EB_2.2

In [13]:
participants = os.listdir(task_folders[1])
participants

['eb_messages.log',
 'Q248_P.edf',
 'Q248_P_warning.log',
 'Q248_P_error.log',
 'actual_TASK_DATASOURCE_DataSource_Q1K_ACAR_NSP_pfp_v1_02_BLOCKTASK_DATASOURCE.dat']

In [14]:
# Define root directory for all eye-tracking tasks 
root_dir = '../../../../Sharing/CHUSJ-Q1K-PILOT/experimental/'

# Define the output directory for the processed data
output_dir = f'../../sourcedata/{site}/et/'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Track participants with missing EEG data
missing_eeg = []
et_subjects=[]
transformed_et=[]
test_missing=[]
#versions = ["version_warnings_on", "version_warnings_off", "version_warnings_off/version_warnings_off"]
versions = [""]
# Iterate thourgh each site
#for site in sites: 
print(site)
print(task_id_in)
#if site == "HSJ":
for version in versions: 
    #et_dir= os.path.join(root_dir, f"{site}/sourcedata/et/Q1K_EB_2.2.299/", f"{version}/")
    et_dir= os.path.join(root_dir, f"{site}/sourcedata/et/Q1K*.2.299/")
    print(et_dir)
    # Iterate through each task
    #for task in tasks:
    task_folders = glob.glob(os.path.join(et_dir, f"*_{task_id_in}_*/results/"))
    print(task_folders)
    for task_folder in task_folders:
        print(task_folder)
        # Iterate over participant folders in current task folder 
        participants = os.listdir(task_folder)
        for participant in participants:
            print(participant)
            participant_folder = os.path.join(task_folder, participant)
            # print(participant_folder)
            et_subjects.append(participant)

            # Process .edf file 
            
            # Create new partcipant name based on Q1K ID
            transformed_id= eb_id_transform(participant) # Transform the ID to match the EEG ID
            transformed_et.append(transformed_id) # Append the transformed ID to the list
            if transformed_id in eeg_q1k_subjects_df.et_ID.values: # Check if the transformed ID is in the EEG ID list
                print('booya')
                print(transformed_id)
                new_participant = eeg_q1k_subjects_df.loc[eeg_q1k_subjects_df['et_ID'] == transformed_id].q1k_ID.values[0]
                print(new_participant)
                final_output_dir = output_dir
                if not os.path.exists(os.path.join(final_output_dir, new_participant)):
                    os.makedirs(os.path.join(final_output_dir, new_participant))
            else:
                missing_eeg.append(participant)
                final_output_dir = os.path.join(output_dir + "/archive/missing_eeg/")
                #new_participant = participant
                continue

            if not os.path.exists(os.path.join(final_output_dir, new_participant)):
                    os.makedirs(os.path.join(final_output_dir, new_participant))

        # Process participant files (EDF and TXT)
            process_participant_files(task_id_out, participant, participant_folder, 
                                        transformed_id, new_participant, final_output_dir)
            et_subjects.append(new_participant)
        
#elif site == "MHC":
#        et_dir= os.path.join(root_dir, f"{site}/sourcedata/et/Q1K_EB_2.2.299/")
#        print(et_dir)
#        # Iterate through each task
##        for task in task_id_in:
#        print(task_id_in)
#        task_folders = glob.glob(os.path.join(et_dir, f"*_{task_id_in}_*/results/"))
#        print(task_folders)
#        for task_folder in task_folders:
#            print(task_folder)
#            # Iterate over participant folders in current task folder
#            participants = os.listdir(task_folder)

#            for participant in participants:
#                participant_folder = os.path.join(task_folder, participant)
#                print(participant_folder)
#                et_subjects.append(participant)
#                # Process .edf file 
#                # Create new partcipant name based on Q1K ID
#                transformed_id= eb_id_transform(participant)
#                transformed_et.append(transformed_id)
#                if transformed_id in eeg_q1k_subjects_df.et_ID.values:
#                    new_participant = eeg_q1k_subjects_df.loc[eeg_q1k_subjects_df['et_ID'] == transformed_id].q1k_ID.values[0]
#                    final_output_dir = output_dir
#                    #print(new_participant)
#                    if not os.path.exists(os.path.join(final_output_dir, new_participant)):
#                            os.makedirs(os.path.join(final_output_dir, new_participant))
#                else:
#                    missing_eeg.append(participant)
#                    final_output_dir = os.path.join(output_dir + "/archive/missing_eeg/")
#                    new_participant = participant

#                    continue
#                print(new_participant)

#                process_participant_files(task_id_in, participant, participant_folder, 
#                                            transformed_id, new_participant, final_output_dir)
#                et_subjects.append(new_participant)
#                # print(new_participant)



MHC
NSP
../../../../Sharing/CHUSJ-Q1K-PILOT/experimental/MHC/sourcedata/et/Q1K*.2.299/
['../../../../Sharing/CHUSJ-Q1K-PILOT/experimental/MHC/sourcedata/et/Q1K_EB_2.2.299/6_Q1K_ACAR_NSP_pfp_v1_02_deploy/results/', '../../../../Sharing/CHUSJ-Q1K-PILOT/experimental/MHC/sourcedata/et/Q1K_EB_2.2.299/6_Q1K_ACAR_NSP_pfp_v1_deploy/results/']
../../../../Sharing/CHUSJ-Q1K-PILOT/experimental/MHC/sourcedata/et/Q1K_EB_2.2.299/6_Q1K_ACAR_NSP_pfp_v1_02_deploy/results/
CBG
Q248_P
booya
0248P
Q1K_MHC_200248_P
new_file_name: 0248P_NSP.asc
source_path: ../../../../Sharing/CHUSJ-Q1K-PILOT/experimental/MHC/sourcedata/et/Q1K_EB_2.2.299/6_Q1K_ACAR_NSP_pfp_v1_02_deploy/results/Q248_P/Q248_P.edf
destination_path: ../../sourcedata/MHC/et/Q1K_MHC_200248_P/0248P_NSP.asc
edf2asc -y -input ../../../../Sharing/CHUSJ-Q1K-PILOT/experimental/MHC/sourcedata/et/Q1K_EB_2.2.299/6_Q1K_ACAR_NSP_pfp_v1_02_deploy/results/Q248_P/Q248_P.edf ../../sourcedata/MHC/et/Q1K_MHC_200248_P/0248P_NSP.asc

EDF2ASC: EyeLink EDF file -> AS

In [ ]:
# Save the list of participants with eyetracking data but missing EEG data inside teh archive folder
missing_eeg_df = pd.DataFrame({'participants': list(set(missing_eeg))})
missing_eeg_df.to_csv(os.path.join(output_dir, 'archive/missing_eeg/q1k_missing_eeg.csv'), index=False)

In [ ]:
# Save look up table
eeg_q1k_subjects_df.to_csv(os.path.join(output_dir, 'archive/missing_eeg/et_eeg_lookup_table.csv'), index=False)